# Step 1 - Scraping

In [12]:
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup
import time
import pandas as pd
import requests

In [13]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

# NASA Mars News

In [14]:
# Scrape the [NASA Mars News Site](https://mars.nasa.gov/news/) and collect the latest News Title and Paragraph Text. 
# Assign the text to variables that you can reference later.

url = "https://mars.nasa.gov/news/"
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
article = soup.find("div", class_="list_text")
news_p = article.find("div", class_="article_teaser_body").text
news_title = article.find("div", class_="content_title").text
print(news_title)
print(news_p)

NASA's MAVEN Spacecraft Finds That "Stolen" Electrons Enable Unusual Aurora on Mars
Auroras appear on Earth as ghostly displays of colorful light in the night sky, usually near the poles.


# JPL Mars Space Images - Featured Image

In [15]:
# Visit the url for JPL Featured Space Image [here](https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars).
# Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called `featured_image_url`.
# Make sure to find the image url to the full size `.jpg` image.
# Make sure to save a complete url string for this image.

url = "https://jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)

# Scrape the browser into soup and use soup to find the image of mars
# Save the image url to a variable called `img_url`
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
image = soup.find("img", class_="thumb")["src"]
#image = soup.find("article", class_="carousel_item")["style"]
img_url = "https://jpl.nasa.gov"+image
featured_image_url = img_url

# Use the requests library to download and save the image from the `img_url` above
import shutil
response = requests.get(img_url, stream=True)
with open('img.jpg', 'wb') as out_file:
    shutil.copyfileobj(response.raw, out_file)
    
# Display the image with IPython.display
from IPython.display import Image
Image(url='img.jpg')    


# Mars Weather

In [16]:
# visit the mars weather report twitter and scrape the latest tweet
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)
html = browser.html
mars_weather_soup = BeautifulSoup(html, 'html.parser')

tweets = mars_weather_soup.find('ol', class_='stream-items')
mars_weather = tweets.find('p', class_="tweet-text").text
print(mars_weather)

L-2 years. #Mars2020


# ### Mars Facts

In [17]:
# visit space facts and scrape the mars facts table
request_mars_space_facts = requests.get("https://space-facts.com/mars/")

mars_space_table_read = pd.read_html(request_mars_space_facts.text)

mars_space_table_read

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [18]:
df = mars_space_table_read[0]
df.set_index(0, inplace=True)

mars_data_df = df
mars_data_df

,1
0,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [19]:
mars_data_html = mars_data_df.to_html()
mars_data_html

mars_data_html.replace('\n', '')

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>1</th>    </tr>    <tr>      <th>0</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.52 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-153 to 20 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

In [20]:
# save to HTML table
mars_data_df.to_html('mars_table.html')

# Mars Hemispheres

In [21]:
# scrape images of Mars' hemispheres from the USGS site
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
mars_dicts = []

for i in range(1,9,2):
    mars_dict = {}
    hemi_dict = {}
    
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    name_links = soup.find_all('a', class_='product-item')
    title = name_links[i].text.strip('Enhanced')
    
    detail_links = browser.find_by_css('a.product-item')
    detail_links[i].click()
    browser.find_link_by_text('Sample').first.click()
    browser.windows.current = browser.windows[-1]
    img_html = browser.html
    browser.windows.current = browser.windows[0]
    browser.windows[-1].close()
    
    img_soup = BeautifulSoup(img_html, 'html.parser')
    img_url = img_soup.find('img')['src']

    hemi_dict['title'] = title.strip()

    hemi_dict['img_url'] = img_url

    mars_dicts.append({"Title": title, "Image_Url": img_url})


    print(mars_dicts)
    print(hemi_dict)

[{'Title': 'Cerberus Hemisphere ', 'Image_Url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}]
{'title': 'Cerberus Hemisphere', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}
[{'Title': 'Cerberus Hemisphere ', 'Image_Url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'Title': 'Schiaparelli Hemisphere ', 'Image_Url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}]
{'title': 'Schiaparelli Hemisphere', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}
[{'Title': 'Cerberus Hemisphere ', 'Image_Url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'Title': 'Schiaparelli Hemisphere ', 'Image_Url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparell

In [23]:
    print(mars_dicts)

    

[{'Title': 'Cerberus Hemisphere ', 'Image_Url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'Title': 'Schiaparelli Hemisphere ', 'Image_Url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'Title': 'Syrtis Major Hemisphere ', 'Image_Url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'Title': 'Valles Marineris Hemisphere ', 'Image_Url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]
